In [ ]:
!pip3 install gym --user
!pip3 install numpy --user
!pip3 install matplotlib --user

!pip3 install box2d --user
!pip3 install pyvirtualdisplay -U

# Q-learning

Dans le précédent workshop nous avions vue les bases du **Reinforcement learning** en resolvant L-Antique Maze via la value function. <br>
Ça été l'occasion de découvrir les diverses notions mathematiques derriere le RL (MDP, VF, etc)

Dans ce workshop, vous allez apprendre les bases de la notion du **Q-learning** et decouvrir les environements Gym.<br>
Vous allez pour cela à la fin de ce workshop résoudre un environement nommé [MontainCar](gym.openai.com/envs/MountainCar-v0/).

### Packages
Importons dans un premier temps les dépences suivantes:  
-numpy est le package fondamental pour le calcul scientifique avec Python.  
-matplotlib  est une librairie connue pour afficher des graphiques en Python.  
-[gym](https://pypi.org/project/gym/0.7.4/) est un tool utils lors d'usage de reinforcement learning

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt

from validation_tests import *
from pyvirtualdisplay import Display
%matplotlib inline

# 1 - Prise en main avec l'environnement

Avant de commencer l'implémenter du Q-learning, commençons par nous familiariser avec la librairie `gym`.
Gym vous permet de tester votre agent dans un environement. Les environements fournits sont variés et de divers complexités.

Celui d'aujourd'hui est nommé `MountainCar-V0`, il consiste en un vehicule situé au creux d'une coline ayant pour but de la franchir.<br>

Commençons déjà par charger et afficher notre environement:

In [ ]:
env = gym.make('MountainCar-v0')

env.seed(1)
env.reset()

img = plt.imshow(env.render(mode='rgb_array'))
env.render()
env.close()

Tres bien, nous avons afficher notre environement.

Quelques explications:
-  `gym.make()` nous a permit de charger notre environement
-  `env.seed()` nous permet d'avoir les memes resultats
-  `env.reset()` réinitialise l'environement et return le stat de votre env
-  `env.render()` vous permet d'afficher notre env
-  `env.close()` ferme l'environement

Maintenant voyons comment peut-on interagir avec:

In [ ]:
env = gym.make('MountainCar-v0')

env.seed(1)
env.reset()

img = plt.imshow(env.render(mode='rgb_array'))
for j in range(50):
    env.render()
    action = env.action_space.sample()
    state, reward, done, _ = env.step(action)
    if done:
        break 
env.close()

Voila qui est plus interessant.

Quelques explications:
-  `env.action_space.sample()` prend une action au hasard parmit celles possible
-  `state` est votre etat apres l'action effectué
-  `reward` est la récompense reçu pour avoir effectué cette action
-  `done` indique si l'environement est terminé

Le `state` représente l'état de votre agent dans son environement, dans le cas de l'environement *MountainCar-v0* le state est composé de deux valeurs: `position` et `velocity`.<br>
Comme leurs nom l'indique, `position` représente la position de l'agent dans l'environement et `velocity` represente sa velocité à un instant $t$.

Regardons comment retrouver ses informations:

In [ ]:
print("Etat: ", env.observation_space)

print("low: ", env.observation_space.low)
print("hight: ", env.observation_space.high)

Decortiquons ces informations ensembles.

`Box()` represent un talbeau à N dimensions, il est ici de dimension $(2,)$ ce qui signifie que notre `state` est de dimsensions $(2,)$.<br>
Cela correspond bien à (`position`, `velocity`) comme nous l'avons dit precedemment.

On a aussi print les `low` et `hight` de ces deux valeurs, ce sont les deux valeurs extremes que peut avoir notre `state`.

`[-1.2, 0.6]` correspond à l'encradrement des valeurs de `position`.<br>
`[-0.07, 0.07]` correspond à l'encradrement des valeurs de `velocity`.

Pour en savoir plus vous pouvez voir le repo github de l'environement [MountainCar](https://github.com/openai/gym/wiki/MountainCar-v0).

Maintenant que nous savons comment interpreter les informations de `state`, passons au controle de notre agent.

**Exercice:** Affichez l'espace d'action de notre agent dans son environement.<br>
**Indice:** [doc de Gym](https://gym.openai.com/docs/)

In [ ]:
print("Action: ", )  # rajoutez votre code (~1 ligne)

**Resultat attendu:** `Action:  Discrete(3)`

Interpretons ces informations:

`Discrete()` signifie que toutes nos actions $\in N+$.<br>
Le `3` correspond aux nombre d'actions effectuables par notre agent dans son environement, dans notre cas ces actions sont: $Reculer$, $Attendre$ et $Avancer$.

# 2 - Le Q-learning


<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/678cb558a9d59c33ef4810c9618baf34a9577686">

### Nomenclature:

- $Q$ est notre `Q-table`
- $St$ représente notre state à un instant $t$
- $at$ représente l'action prise à un instant $t$
- $\alpha$ (alpha) est le learning rate, c'est l'importance que l'on donne au new state. <br>Sachant 0 < $\alpha$ < 1, plus de learning rate est élevé, plus on donne de l'importance au new state.
- $\gamma$ (gamma) est le discount factor, c'est l'importance donné à un futur reward. <br>Sachant 0 < $\gamma$ < 1, plus le discount factor est élevé, plus notre agent "pensera" au long terme.
- $\epsilon$ (epsilon) est le facteur aleatoire de l'apprentissage<br>Sachant 0 < $\epsilon$ < 1, plus epsilon est élevé, plus notre agent effectura des actions aleatoires.


### Algorithme du Q-learning

Pour commencer l'uttilisateur definit sa Q-table à des valeur arbitraires.<br>

Pour rappel la Q-table contient pour chaque binomes state $S$ et action $a$ le reward qui leur est associé.<br>
Si l''uttilisateur se trouve dans un state $St$ et effectue une action $at$ il recevra en reward $Q[St, at]$.

Une fois la Q-table créée on commence l'algorithme:

L'uttilisateur se trouve à un state $S$.<br>Il recupere une nombre random $r$, si $r$ < $\epsilon$ il effectue une action aleatoire, si $r$ > $\epsilon$ il effectue alors une action dite greedy.<br>
Apres avoir effectué l'action $a$, l'uttilisateur se trouve dans un state $S'$ et a reçu un reward $r$.


L'uttilisateur va pouvoir updater sa Q-table, pour rappel:
$Qnew[st, at] = Q[st, at] + \alpha * (rt  + \gamma  * maxQ(st+1, a) - Q[st, at])$

Au fur et à mesur que l'uttilisateur update sa Q-table, l'agent sera de mieux en mieux comment optimiser ses action pour obtenir le meilleur reward et ainsi résoudre son environement.

### La pratique 

Nous avons vue la theorie, maintenant passons a la pratique.


Commençons par initialiser `states` qui contiendra tout les `states` que notre agent pourra recontrer dans son environement.

Pour rappel:
- `[-1.2, 0.6]` correspond à l'encradrement des valeurs de `position`.<br>
- `[-0.07, 0.07]` correspond à l'encradrement des valeurs de `velocity`.
- Notre agent obtient -1 à chaque action.
- Moins notre agent effectue d'action, plus son reward sera grand.

Une question se pose alors:<br>Il y a une infinité de valeurs dans l'encadrement `[-1.2, 0.6]` et l'encadrement `[-0.07, 0.07]`. Comment toutes les stocker ?<br>

Nous ne pouvons juste pas c'est pourquoi nous alons devoir se contenter d'un echantillons de ces valeurs.<br>
La taille de cette echantillon est variable selon la situation, dans notre cas nous allons choisir un echantillon de taille $20$.

La taille de notre echantillon est un vacteur important dans le cas du Q-Learning.
Plus l'echantillons est grand, plus notre agent sera precis mais plus il prendra du temps à tout explorer.
Il y a aussi le facteur memoir qui rentre en jeu, la taille alloué à notre Q-table augmente de façon exponentiel:
- Un echantillon de taille 10 pour 3 actions possibles menera à une Qtable de shape `((10, 10), 3)` et contenant $300$ valeurs.
- Un echantillon de taille 15 pour 3 actions possibles menera à une Qtable de shape `((15, 15), 3)` et contenant $675$ valeurs.
- Un echantillon de taille 20 pour 3 actions possibles menera à une Qtable de shape `((20, 20), 3)` et contenant $1200$ valeurs.

Ici doubler la taille de l'echantillons ne double donc pas seulement la taille de notre Q-table mais la quadruple !

Dans notre cas voici à quoi ressemble notre echantillon:

In [ ]:
STATES_SPACE = 20

POSITION_SPACE = np.linspace(-1.2, 0.6, STATES_SPACE)
VELOCITY_SPACE = np.linspace(-0.07, 0.07, STATES_SPACE)

print(f"POSITION_SPACE: {POSITION_SPACE}\n")
print(f"VELOCITY_SPACE: {VELOCITY_SPACE}\n")

La solution la plus evidante maintenant serai de stocker dans `states[0][0]` la valeur `(-1.2)` et dans `states[0][1]`la valeur `(0.07)` etc. mais cette solution apporterai d'autre soucis.<br>
Exemple: On se retrouverai à avoir dans `states[1][0]` la valeur `-1.10526316` mais notre env ne nous retournera probablement pas la valeur exact `-1.10526316` ce qui demanderai a chaque fois d'arrondire cette derniere.

Pour contrer cela au lieux de stocker dans `states` les valeurs de notre echantillons, nous allons stocker les index correspondant aus valeurs de notre echantillons.<br>
Exemple: `states[0]` contiendra alors `(0, 0)` correspondant à respectivement $-1,2$ et $-0.07$ dans notre echantillon.

**Exercices:** Completez la fonction `init_states()` pour qu'elle return l'array `states` de taille `STATES_SPACE` contenant toutes les combinaisons des valeurs representants notre echantillon `(observation, velocity)`.<br>
**Indices:**
- Uttilisez `STATES_SPACE`
- `state[1]` correspondant à `(0, 1)` représente la $1ere$ valeur de notre echantillon de `observation` et la $2eme$ valeur de notre echantillon de `velocity`.
- `state[20]` correspondant à `(1, 0)` représente la $2eme$ valeur de notre echantillon de `observation` et la $1ere$ valeur de notre echantillon de `velocity`.

In [ ]:
def init_states():
    states = []
    # rajoutez votre code (~3 lignes)

    
    
    # fin de votre code
    return states

assert valide_init_states(init_states(), STATES_SPACE), "Provided function does not match requirements"

**Exercice:** Completez la fonction `get_indexes()` pour que depuis un `state` donné en argument elle return les indexs correspondants dans notre echantillon.<br>
**Indices:** `np.digitize()`

In [ ]:
def get_indexes(state):
    # rajoutez votre code (~2 lignes)
    
    
    # fin de votre code
    return (position_index, velocity_index)
  
assert valide_get_indexes(get_indexes((0, 0)), POSITION_SPACE, VELOCITY_SPACE), "Provided function does not match requirements"

Passons maintenant à la gestion des actions de notre agent.

Lorsque l'on demande à notre agent d'effectué une action qui selon lui est optimal est effectu alors une action dite "*greedy*".<br>
C'est justement ce que vous allez implémenter.

**Exercice:** Completez la fonction `greedy_step()` pour que depuis un `state` donné elle return l'action oprimal à faire.

In [ ]:
ACTION_SPACE = [0, 1, 2]

def greedy_step(Q, state, actions=[0, 1, 2]):
    # rajoutez votre code (~2 lignes)
    
    
    # fin de votre code
    return action
  
assert valide_greedy_step(create_testing_Qtable()) == greedy_step(create_testing_Qtable(), (1, 2)), "Provided function does not match requirements"

Implementons maintenant l'usage d'epsilone.

**Exercice:** Completez la fonction take_action pour qu'elle return l'action à faire à un moment $t$.<br>
**Indices:**
- Pour $r$ un nombre aleatoire $\in$ `ACTION_SPACE`, si $r$ > $\epsilon$ alors l'action effectué sera aleatoire.
- `np.random.random()`

In [ ]:
def take_action(epsilon, Q, state, actions=[0, 1, 2]):
    # rajoutez votre code (~3 lignes)
    
    
    
    # fin de votre code

Vous pouvez maintenant passer à l'initialisation de votre Q-table.

**Exercice:** Completez la fonction `init_Qtable() ` pour qu'elle return votre Q-table avec toutes ses valeurs initialisés à $0$.<br>
**Indice:**
- Votre Q-table est de shape `((STATE_SPACE, STATE_SPACE), ACTION_SPACE)`
- Qu'est ce qui est de shape `(STATE_SPACE, STATE_SPACE)` ?

In [ ]:
def init_Qtable(states, action_space):
    Q = {}
    # rajoutez votre code (~3 lignes)
    
    
    
    # fin de votre code
    return Q
  
assert valide_init_Qtable(init_states(), ACTION_SPACE) == init_Qtable(init_states(), ACTION_SPACE), "Provided function does not match requirements"

Maintenant que votre Q-table est initalisé vous devez l'updater.

**Exercice:** Completez `Qfunction` pour qu'elle return la nouvelle valeur de `Q[state, action` selon la *Q-function*.<br>
**Indice:** $Qnew[st, at] = Q[st, at] + \alpha * (rt  + \gamma  * maxQ(st+1, a) - Q[st, at])$

In [ ]:
def Qfunction(state, action, reward, new_state, alpha, gamma):
  max_action = greedy_step(Q, new_state)
  
  return Q[state, action] + alpha * (reward + gamma * Q[new_state, max_action] - Q[state, action])

# Mise en situation

Tout est en place pour passer à la mise en siutation (ou presque, nous y reviendrons).

In [ ]:
NUMBER_OF_GAMES = 2_000
ALPHA = 0.1
GAMMA = 0.99
epsilon = 1.0
EPS_MIN = 0.1
EPS_DECAY = 0.995

env = gym.make('MountainCar-v0')

env.seed(1)
np.random.seed(1)

episode_score = 0
total_rewards = np.zeros(NUMBER_OF_GAMES)
memory = []

states = init_states()
Q = init_Qtable(states, ACTION_SPACE)

**Exercice:** Completez le code ci-dessous.

In [ ]:
for game_number in range(NUMBER_OF_GAMES):
    done = False
    obs = env.reset()
    state = get_indexes(obs)
    if game_number % 100 == 0 and game_number > 0:
        print('episode ', game_number, 'score ', episode_score, 'epsilon %.3f' % epsilon)
    episode_score = 0
    while not done:
        if (game_number % 500 == 0):
            env.render()
        # rajoutez votre code (~5 lignes)
        
        
        
        
        
        # fin de votre code
        episode_score += reward
    total_rewards[game_number] = episode_score
    epsilon = max(EPS_MIN, epsilon * EPS_DECAY)

print(f"average reward: {sum(total_rewards) / len(total_rewards)}")

In [ ]:
mean_rewards = np.zeros(NUMBER_OF_GAMES)

for t in range(NUMBER_OF_GAMES):
    mean_rewards[t] = np.mean(total_rewards[int(max(0, t - NUMBER_OF_GAMES / 10)): t + 1])

plt.plot(mean_rewards)

Super, on voit que notre agent apprend bien à resoudre son environement mais n'y a-t-il pas un moyen d'optimiser son apprentissage ?<br>

Si et vous avez deja vue comment: en uttilisant le principe d'*experience replay*.<br>
On va rajouter une memory à notre agent pour qu'il puisse s'entrainer plusieurs fois sur des situations qu'il pourrait rencontrer que rarement.

**Exercice:** Completez la fonction `add_memory` pour qu'a chaque appel elle update memory avec la nouvelle value donné en argument.<br>
**Indices:**
- Attention à ne pas depassé `memory_size`
- Si on doit remplacer un valeur, on remplace la plus ancienne

In [ ]:
def add_memory(memory, state, action, reward, new_state, memory_size=600):
    # rajoutez votre code (~3 lignes)
    
    
    
    
    # fin de votre code
    return memory
  
def train_on_memory(memory, Q, alpha, gamma):
    for mem in memory:
        Q[mem['state'], mem['action']] = Qfunction(mem['state'], mem['action'], mem['reward'], mem['new_state'], alpha, gamma)
    return Q

In [ ]:
epsilon = 1.0
episode_score = 0
total_rewards = np.zeros(NUMBER_OF_GAMES)

Q = init_Qtable(states, ACTION_SPACE)
memory = []

**Exercice:** Completez le code ci-dessous pour que votre agent uttilise `memory`.

In [ ]:
for game_number in range(NUMBER_OF_GAMES):
    done = False
    obs = env.reset()
    state = get_indexes(obs)
    if game_number % 100 == 0 and game_number > 0:
        print('episode ', game_number, 'score ', episode_score, 'epsilon %.3f' % epsilon)
    episode_score = 0
    while not done:
        if (game_number % 500 == 0):
            env.render()
        # rajoutez votre code (~5 ligne)
        
        
        
        
        
        # fin de votre code
        episode_score += reward
    # rajoutez votre code (~1 ligne)
    
    # fin de votre code
    total_rewards[game_number] = episode_score
    epsilon = max(EPS_MIN, epsilon * EPS_DECAY)

print(f"average reward: {sum(total_rewards) / len(total_rewards)}")

In [ ]:
mean_rewards = np.zeros(NUMBER_OF_GAMES)

for t in range(NUMBER_OF_GAMES):
    mean_rewards[t] = np.mean(total_rewards[:])
    mean_rewards[t] = np.mean(total_rewards[int(max(0, t - NUMBER_OF_GAMES / 10)): t + 1])

plt.plot(mean_rewards)

Félicitation vous avez réussis votre premier Q-Learning !

Essayez de faire de même pour [cette environement](https://gym.openai.com/envs/CartPole-v1/) 😉